In [64]:
# %pip install -U langgraph langchain-community langchain-anthropic tavily-python langgraph-checkpoint-sqlite

In [1]:
import pandas as pd

In [2]:
categories = pd.read_csv('data/amazon_categories.csv')
products = pd.read_csv('data/amazon_products.csv')

In [3]:
faq = pd.read_csv('data/ecommerce_faq.csv')

In [5]:
products.columns

Index(['asin', 'title', 'imgUrl', 'productURL', 'stars', 'reviews', 'price',
       'listPrice', 'category_id', 'isBestSeller', 'boughtInLastMonth'],
      dtype='object')

In [6]:
products = products.replace(0, '')

In [7]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1426337 entries, 0 to 1426336
Data columns (total 11 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   asin               1426337 non-null  object
 1   title              1426336 non-null  object
 2   imgUrl             1426337 non-null  object
 3   productURL         1426337 non-null  object
 4   stars              1426337 non-null  object
 5   reviews            1426337 non-null  object
 6   price              1426337 non-null  object
 7   listPrice          1426337 non-null  object
 8   category_id        1426337 non-null  int64 
 9   isBestSeller       1426337 non-null  bool  
 10  boughtInLastMonth  1426337 non-null  object
dtypes: bool(1), int64(1), object(9)
memory usage: 110.2+ MB


In [8]:
categories = categories.rename(columns={
'id':'category_id'
})

In [9]:
df = pd.merge(products, categories, on='category_id')  # Default is inner join

In [10]:
df.dropna(inplace=True)

In [21]:
df.head(1)

,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,category_id,isBestSeller,boughtInLastMonth,category_name
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",https://m.media-amazon.com/images/I/815dLQKYIY...,https://www.amazon.com/dp/B014TMV5YE,4.5,,139.99,,104,False,2000,Suitcases


# Clip Embedding

In [23]:
import pandas as pd
import os
from PIL import Image
image_paths = [img for img in df["imgUrl"]]

In [25]:
len(image_paths)

1426336

In [ ]:
df['']

In [27]:
import torch
import clip
from tqdm import tqdm  
from torchvision import transforms

# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)


print(device)

cuda


In [35]:
%cd "C:\\Users\\emann\\Documents\\GitHub\\MultiModal-E-Commerce-Customer-Support-Chatbot"

C:\Users\emann\Documents\GitHub\MultiModal-E-Commerce-Customer-Support-Chatbot


C:\Users\emann\anaconda3\envs\nst\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [37]:
%pwd

'C:\\Users\\emann\\Documents\\GitHub\\MultiModal-E-Commerce-Customer-Support-Chatbot'

In [46]:
img_urls = df['imgUrl'][].tolist()

img_embeddings = []
products_url = []

for url in img_urls:
  try:
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert("RGB")
    img_tensor = preprocess(img).unsqueeze(0).to(device)

    with torch.no_grad():
      embeddings = model.encode_image(img_tensor).cpu().numpy().flatten()

    img_embeddings.append(embeddings)
    products_url.append(url)

  except Exception as e:
        print(f"Error processing {url}: {e}")  # Skip failed images

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
embeddings = []

for img_path in tqdm(image_paths, desc="Extracting Embeddings", unit="image"):
    try:
        embeddings.append(get_image_embedding(img_path))
    except Exception as e:
        print(f"Skipping {img_path}: {e}")  # Just log and move on

---
---

In [71]:
!pip install -U langchain-groq


In [3]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
import os
from langchain_groq import ChatGroq

In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.memory import ConversationBufferMemory

In [5]:
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\emann\AppData\Local\Temp\ipykernel_8256\3414252995.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [43]:
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
faq_texts = [f"FAQ: {row['Question']} Answer: {row['Answer']}" for _, row in faq.iterrows()]
product_texts = [f"Product: {row['title']} Price: ${row['price']} Category: {row['category_name']} Image: {row['imgUrl']}" for _, row in df.iterrows()]

all_texts = faq_texts + product_texts

In [55]:
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
faq_texts = [f"FAQ: {row['Question']} Answer: {row['Answer']}" for _, row in faq.iterrows()]
product_texts = [f"Product: {row['title']} Price: ${row['price']} Category: {row['category_name']} Image: {row['imgUrl']}" for _, row in df.iterrows()]

all_texts = faq_texts + product_texts

documents = [Document(page_content=text) for text in all_texts]

faiss_index = FAISS.from_documents(documents, embed_model)
faiss_index.save_local("faiss_index")

In [8]:
faiss_index = FAISS.load_local("faiss_index", embed_model, allow_dangerous_deserialization=True)
query = "I want a teady bear"
docs = faiss_index.similarity_search(query, k=1)
retrieved_info = docs[0].page_content if docs else "No relevant information found."
print(retrieved_info)

Product: Care Bears 22077 14 Inch Medium Plush Togetherness Bear, Collectable Cute Plush Toy, Cuddly Toys for Children, Soft Toys for Girls and Boys, Cute Teddies Suitable for Girls and Boys Aged 4 Years + Price: $ Category: Stuffed Animals & Plush Toys Image: https://m.media-amazon.com/images/I/71bCBVDTyGS._AC_UL320_.jpg


In [11]:
# os.environ["GROQ_API_KEY"] = "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"
# llm = ChatGroq(model_name="llama-3-70b-versatile", temperature=0)

In [ ]:
def get_conversation_chain(vectorDB):
    llm = ChatGroq(api_key='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',
                   model_name = 'llama3-70b-8192', temperature = 0)

    prompt_shopping_assistant = """
    *Welcome Message:*
    "Hi there! I'm your Shopping Assistant. I'll be happy to help in your shopping journey. How may I assist you today?"
    *Your role* ".
    
    *User Interaction:*
    - *Query Received*: {input}
    - *Your Database*: {context}
        Use this as your primary knowledge base.

    *Guidelines for Usage of Database:*
    
    1. I operate strictly within the bounds of our product database. If an inquiry is about topics outside our catalog, I will suggest alternatives or ask for more details about our products.
       For instance: "I can only assist with questions about our products. Can you specify which one you’re interested in?"
    2. If the context is irrelevant based on the user query or if the required product information is missing, don't refer to the mismatched context or make assumptions.
       Instead, directly ask the user to provide more specific details or shift their inquiry to a different product that is available in our database.
    3. When the context you get does not align with the user's query or if the required product information is not in our database, clearly state:
       "I apologize, but I don't have any information on that particular product. Could you please specify another product or feature you are interested in?"
       This keeps the conversation focused and encourages users to ask about other products we might have information on.
       
    *Strict Restrictions*:
        - I can only use the provided knowledge base to answer queries.
        - I am not allowed to provide any support except for product-related queries.
        - I will refuse any non-product-related queries without explanation.
        - I will not attempt to answer questions outside my product-related scope.
        - I will provide concise and direct responses.
        - I will strictly adhere to all the above guidelines.

    *Execution Standards*:
        - My responses will be succinct, precise, and tailored to your specific inquiries.
        - I will maintain a professional tone throughout our interaction.
        - I will use proper text formatting for better readability."""
    
    prompt_chatbot = ChatPromptTemplate.from_messages([('system', prompt_shopping_assistant),
                     MessagesPlaceholder('chat_history'), ('human', '{input}')])

    context_retriever = create_history_aware_retriever(llm,
                        vectorDB.as_retriever(), prompt_chatbot)

    document_chain = create_stuff_documents_chain(llm, prompt_chatbot)
    conversation_chain = create_retrieval_chain(context_retriever, document_chain)
    memory = ConversationBufferMemory(return_messages = True, memory_key = 'chat_history',
                                      output_key = 'result', input_key = 'input')
    return conversation_chain, memory

def main(input_dict):
    vectorDB_path = input_dict['vectorDB_path']
    vectorDB = FAISS.load_local(vectorDB_path,
               HuggingFaceEmbeddings(model_name = 'all-MiniLM-L6-v2'),
               allow_dangerous_deserialization = True)

    conversation_chain, memory = get_conversation_chain(vectorDB)
    while True:
        user_input = input('User: ')

        documents_retrieved = vectorDB.similarity_search(user_input, threshold=0.9)
        if not documents_retrieved:
            response = "I apologize, but I don't have any information on that particular product. Could you please specify another product or feature you are interested in?"
        else:
            context = " ".join([doc.page_content for doc in documents_retrieved if 'page_content' in doc])
        chain_input = {'input': user_input, 'chat_history': memory.chat_memory.messages, 'context': context}

        result = conversation_chain.invoke(chain_input)
        response = result['answer']
        print('-'*80)
        print('Assistant: ', response)
        print('-'*80)
        memory.save_context({'input': user_input}, {'result': response})

input_dict = {'vectorDB_path': 'faiss_index'}
main(input_dict) 

C:\Users\emann\AppData\Local\Temp\ipykernel_8256\2202291290.py:46: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages = True, memory_key = 'chat_history',


User:  Recommend a phone under $500


--------------------------------------------------------------------------------
Assistant:  Based on our product database, I'd like to recommend the **VTech IS8151-5 Super Long Range 5 Handset DECT 6.0 Cordless Phone for Home**. It's a great option that fits your budget of under $500, priced at $136.20.

This phone features a super long range of 2300 ft, an answering machine, call blocking, Bluetooth, a headset jack, power backup, intercom, and is expandable to 12 handsets. It's a great value for its price and has received positive reviews from customers.

Would you like to know more about this product or explore other options within your budget?
--------------------------------------------------------------------------------


User:  can i see the image as well?


--------------------------------------------------------------------------------
Assistant:  I apologize, but I don't have the specific product "VTech IS8151-5 Super Long Range 5 Handset DECT 6.0 Cordless Phone for Home" in my database. However, I can suggest some other options that fit your budget of under $500.

Here are a few products from my database:

* **VTech CS6529-4B 4-Handset DECT 6.0 Cordless Phone with Answering System and Caller ID**: This phone is priced at $86.79 and has the following image: https://m.media-amazon.com/images/I/71+kzN3iLeL._AC_UL320_.jpg
* **VTech CS6124 DECT 6.0 Cordless 2K DCI 1080p Phone with Answering System and Caller ID/Call Waiting**: This phone is priced at [no price available] and has the following image: https://m.media-amazon.com/images/I/81+pnSPxKAL._AC_UL320_.jpg
* **VTech VG101-11 DECT 6.0 Cordless Phone for Home**: This phone is priced at $18.45 and has the following image: https://m.media-amazon.com/images/I/71imBgvdhYL._AC_UL320_.jpg

Ple

...

--------------------------------------------------------------------------------
Assistant:  I apologize for the confusion! Based on your original query, I see that we have an iPhone option in our database.

Here's a recommendation:

* **Apple iPhone 12 Pro Max, 128GB, Pacific Blue - AT&T (Renewed)**: This iPhone is priced at $510.00 and has the following image: https://m.media-amazon.com/images/I/41ZCg7l8N6L._AC_UL320_.jpg

Please note that this is a renewed iPhone, which means it's a pre-owned device that has been inspected and tested to meet manufacturer standards.

Would you like to know more about this product or explore other iPhone options within your budget?
--------------------------------------------------------------------------------


User:  oh its too much


--------------------------------------------------------------------------------
Assistant:  The Apple iPhone 12 Pro Max is a bit pricey.

Let me suggest a more affordable option:

* **Apple iPhone 11, 64GB, White - Fully Unlocked (Renewed)**: This iPhone is priced at $298.46 and has the following image: https://m.media-amazon.com/images/I/31PY10NqSLL._AC_UL320_.jpg

This is a renewed iPhone 11, which is a more budget-friendly option. Would you like to know more about this product or explore other iPhone options within your budget?
--------------------------------------------------------------------------------


User:  exit


--------------------------------------------------------------------------------
Assistant:  It was nice chatting with you! If you need any assistance in the future, feel free to reach out. Have a great day!
--------------------------------------------------------------------------------


In [42]:
!pip install sentence-transformers openai-clip

In [48]:
import clip
import torch.nn.functional as F
import torch

In [50]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, preprocess = clip.load("ViT-B/32", device=device)

In [ ]:
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Process image
image = Image.open("uploaded_image.jpg")
inputs = processor(images=image, return_tensors="pt", padding=True)
image_embedding = model.get_image_features(**inputs)

# Search for similar products
similar_products = faiss_db.similarity_search(image_embedding, k=5)